In [17]:
from youtube_transcript_api import YouTubeTranscriptApi


In [18]:
def get_youtube_transcript(video_url: str) -> list:
    video_id = video_url.split("v=")[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    return transcript
video_url = "https://www.youtube.com/watch?v=8QfI5a7lTKU"
video_id = video_url.split("v=")[1]

transcript = YouTubeTranscriptApi.get_transcript(video_id)

for segment in transcript:
    print(segment['text'])

what is the common point between all
those robots
well all the robots you see here are
or can be powered by rouse
roth means robot operating system
it is something between a middleware
and a framework built especially for
robotics applications rao's goal
is to provide a standard for robotic
software
that developers can use and reuse in
any robot once you know how to use ros
efficiently you will be able to set up
a new software for a new robot in no
time
you first need to master the basics and
core functionalities
which you will see in this course and
then
for every new robot you program you will
gain more skills
that can apply to other robots
the more you know about ross the easiest
it will be for you to program and
understand
the code of any robot
but is that it no another
philosophy behind ross is don't reinvent
the wheel reinventing the wheel is a
very common problem
in robotics people always start building
new software
that is already existing and thus they
don't have the time
to w

In [19]:
## MongoDB
from pymongo import MongoClient

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client["youtube_database"]  # Database name
collection = db["transcripts"]   # Collection name

# Insert into MongoDB
document = {
    "source": "youtube",
    "video_id": video_id,
    "transcript": transcript
}

collection.insert_one(document)
print("Transcript stored into MongoDB")

Transcript stored into MongoDB


In [20]:
result = collection.find_one({"video_id": video_id})
print(result.keys())

dict_keys(['_id', 'source', 'video_id', 'transcript'])


In [21]:
# Extract text from transcripts
sentences = [i['text'] for i in result['transcript']]
print(sentences)







['what is the common point between all', 'those robots', 'well all the robots you see here are', 'or can be powered by rouse', 'roth means robot operating system', 'it is something between a middleware', 'and a framework built especially for', "robotics applications rao's goal", 'is to provide a standard for robotic', 'software', 'that developers can use and reuse in', 'any robot once you know how to use ros', 'efficiently you will be able to set up', 'a new software for a new robot in no', 'time', 'you first need to master the basics and', 'core functionalities', 'which you will see in this course and', 'then', 'for every new robot you program you will', 'gain more skills', 'that can apply to other robots', 'the more you know about ross the easiest', 'it will be for you to program and', 'understand', 'the code of any robot', 'but is that it no another', "philosophy behind ross is don't reinvent", 'the wheel reinventing the wheel is a', 'very common problem', 'in robotics people always

In [22]:
# Chunk sentences together for embedding
#TODO: Make this chunking better:Maybe capitalize and fins end of sentences in a better way
sentences_in_chunk = 6
chunks = []
for i in range(0, len(sentences), sentences_in_chunk):
    chunks.append(". ".join(sentences[i:i + sentences_in_chunk]))

print(len(chunks))

28


In [23]:
# Embed these chunks
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2') # TODO: Pick better models

embeddings = model.encode(chunks)
embeddings_dict = dict(zip(chunks, embeddings))
print(embeddings.shape)


(28, 384)


In [24]:
# Store embeddings in MongoDB
# Insert into MongoDB
collection = db["embeddings"]   # Collection name
document = {
    "source": "youtube",
    "video_id": video_id,
    "embeddings": embeddings.tolist()
}

collection.insert_one(document)
print("Embeddings stored into MongoDB")


Embeddings stored into MongoDB


In [25]:
# TODO: Do further preprocessing to tokens

In [26]:
## Similarity search using qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from torchvision.transforms import Resize
from tqdm import tqdm
import torch

qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance, for testing, CI/CD

#Create a collection or database of texts where you store the embeddings
my_collection = "text_collection"

qdrant.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
)

# Insert embeddings into Qdrant
for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    qdrant.upsert(
        collection_name=my_collection,
        points=[models.PointStruct(
            id=i,
            vector=embedding,
            payload={"text": chunk}
        )]
    )






In [27]:
qdrant.count(
    collection_name=my_collection,
    exact=True,
)

CountResult(count=28)

In [28]:
## Sanity test
sample_sentence = "What is the meaning of life?"
sample_embedding = model.encode(sample_sentence)
search_result = qdrant.search(
    collection_name=my_collection,
    query_vector=sample_embedding,
    limit=5
)

In [29]:
search_result

[ScoredPoint(id=21, version=0, score=0.1516602337360382, payload={'text': "and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part"}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=10, version=0, score=0.07326249778270721, payload={'text': 'create a robot software. which needs a lot of communication. between its. sub-programs or has some functionalities. that go beyond a very simple use case. etc so for a robot'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=0, version=0, score=0.06109253689646721, payload={'text': 'what is the common point between all. those robots. well all the robots you see here are. or can be powered by rouse. roth means robot operating system. it is something between a middleware'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=22, version=0, score=0.05160970240831375, payload={'text': "i

# LLM

In [30]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 24 GB


In [32]:
from huggingface_hub import login
login()

In [33]:
use_quantization_config = True
model_id = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"

In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 

# 1. Create quantization config for smaller model loading (optional)
# Requires !pip install bitsandbytes accelerate, see: https://github.com/TimDettmers/bitsandbytes, https://huggingface.co/docs/accelerate/
# For models that require 4-bit quantization (use this if you have low GPU memory available)
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

# Bonus: Setup Flash Attention 2 for faster inference, default to "sdpa" or "scaled dot product attention" if it's not available
# Flash Attention 2 requires NVIDIA GPU compute capability of 8.0 or above, see: https://developer.nvidia.com/cuda-gpus
# Requires !pip install flash-attn, see: https://github.com/Dao-AILab/flash-attention 
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 2. Pick a model we'd like to use (this will depend on how much GPU memory you have available)
# model_id = "google/gemma-7b-it"
model_id = model_id # (we already set this above)
print(f"[INFO] Using model_id: {model_id}")

# 3. Instantiate tokenizer (tokenizer turns text into numbers ready for the model) 
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")
# llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id)

if not use_quantization_config: # quantization takes care of device setting automatically, so if it's not used, send model to GPU 
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ


c:\Users\harsh\anaconda3\envs\ai\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\harsh\.cache\huggingface\hub\models--TheBloke--CapybaraHermes-2.5-Mistral-7B-GPTQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
CUDA extension not installed.
CUDA extension not installed.
c:\Users\harsh\anaconda3\envs\ai

In [35]:
llm_model.eval() # put model in evaluation mode
llm_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
  )
  (lm_head): Linear(in_features=4096, out_features=32002, bias=False)
)

In [36]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

262426624

In [37]:
def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.

    See: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822
    """
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 4155817984, 'model_mem_mb': 3963.3, 'model_mem_gb': 3.87}

In [38]:
input_text = "What is ros?"
print(f"Input text:\n{input_text}")


# Create prompt template for instruction-tuned model
dialogue_template = [
    {"role": "user",
     "content": input_text}
]

# Apply the chat template
prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False, # keep as raw text (not tokenized)
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

# # Tokenize the input text
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# print(f"Input IDs:\n{input_ids}")

# prompt = input_text #TODO: Fix a prompt structure?

Input text:
What is ros?

Prompt (formatted):
<|im_start|>user
What is ros?<|im_end|>
<|im_start|>assistant



In [39]:
# Tokenize the input text (turn it into numbers) and send it to GPU
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")

# Generate outputs passed on the tokenized input
# See generate docs: https://huggingface.co/docs/transformers/v4.38.2/en/main_classes/text_generation#transformers.GenerationConfig 
outputs = llm_model.generate(**input_ids,
                             max_new_tokens=256) # define the maximum number of new tokens to create
print(f"Model output (tokens):\n{outputs[0]}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Model input (tokenized):
{'input_ids': tensor([[    1, 32001,  2188,    13,  3195,   349,   712, 28713, 28804, 32000,
         28705,    13, 32001, 13892,    13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Model output (tokens):
tensor([    1, 32001,  2188,    13,  3195,   349,   712, 28713, 28804, 32000,
        28705,    13, 32001, 13892,    13,  1594, 28735, 28725,   442,   272,
         5776,   322,  6692,  1077,  2135, 28725,   349,   264,   808,   302,
         3930, 24133,   304,  7040,  6202,   486,   272,  5776,   322,  1063,
         5926,   438, 28116,  2900, 28723,   661,   349,   396,  1565, 28733,
         1394, 10782,   369,  5312,   272,  4892,  3930, 14461,   354,   272,
         4099, 28725,  2602, 28725,   304,  5225,   302,  7006,  1649, 28723,
           13,    13,  1594, 28735,  5976, 17892,   298,  5061,  2231,  4630,
        28725, 16458, 28725,   304,  9096, 18401,  3930,   486,  7501,   264,


In [40]:
# Decode the output tokens to text
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<s><|im_start|> user
What is ros?<|im_end|> 
<|im_start|> assistant
ROS, or the Robot Operating System, is a set of software libraries and tools developed by the Robotics Group at Stanford University. It is an open-source framework that provides the necessary software infrastructure for the development, control, and operation of robots.

ROS allows developers to easily create complex, robust, and efficient robot software by providing a wide range of functionalities, including hardware driver modules, high-level algorithms, and a variety of tools for building, testing, and deploying robot software. It also facilitates the integration of various hardware components, such as sensors, actuators, and computer vision systems, into a unified software framework.

The main components of ROS include:

1. ROS Core: This includes the fundamental components that enable communication between nodes, such as the ROS message passing system, the ROS parameter server, and the ROS 

In [ ]:
outputs_decoded
clean_outputs

'<s><|im_start|> user\nWhat is ros?<|im_end|> \n<|im_start|> assistant\nROS, or the Robot Operating System, is a set of software libraries and tools developed by the Robotics Group at Stanford University. It is an open-source framework that provides the necessary software infrastructure for the development, control, and operation of robots.\n\nROS allows developers to easily create complex, robust, and efficient robot software by providing a wide range of functionalities, including hardware driver modules, high-level algorithms, and a variety of tools for building, testing, and deploying robot software. It also facilitates the integration of various hardware components, such as sensors, actuators, and computer vision systems, into a unified software framework.\n\nThe main components of ROS include:\n\n1. ROS Core: This includes the fundamental components that enable communication between nodes, such as the ROS message passing system, the ROS parameter server, and the ROS master node.\n

In [41]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")

Input text: What is ros?

Output text:
<s><|im_start|> user
What is ros?<|im_end|> 
<|im_start|> assistant
ROS, or the Robot Operating System, is a set of software libraries and tools developed by the Robotics Group at Stanford University. It is an open-source framework that provides the necessary software infrastructure for the development, control, and operation of robots.

ROS allows developers to easily create complex, robust, and efficient robot software by providing a wide range of functionalities, including hardware driver modules, high-level algorithms, and a variety of tools for building, testing, and deploying robot software. It also facilitates the integration of various hardware components, such as sensors, actuators, and computer vision systems, into a unified software framework.

The main components of ROS include:

1. ROS Core: This includes the fundamental components that enable communication between nodes, such as the ROS message passing system, the ROS parameter serve

In [42]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "WWhat are the main differences between ROS 1 and ROS 2?",
    "Explain the concept of topics, services, and actions in ROS.",
    "What is the purpose of the catkin_make command in ROS 1?",
    "What is the function of rviz in ROS?",
    "What is the purpose of rqt_graph, and how can it help in debugging a ROS system?"
] 

# Manually created question list
manual_questions = [
    "What is ros?",
]

query_list = gpt4_questions + manual_questions

In [43]:
import random
query = random.choice(query_list)

print(f"Query: {query}")



Query: What is ros?


In [44]:
search_result = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)

In [52]:
search_result[0].payload['text']

"and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part"

## Augment Similarity Search and LLM output 

In [46]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # Create a base prompt with examples to help the model
    # Note: this is very customizable, I've chosen to use 3 examples of the answer style we'd like.
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""

    # Update base prompt with context items and query   
    base_prompt = base_prompt.format(context=context, query=query)

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [53]:
# import random
query = random.choice(query_list)
print(f"Query: {query}")

# Get relevant resources
# scores, indices = retrieve_relevant_resources(query=query,
#                                               embeddings=embeddings)
search_results = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)
    
# Create a list of context items
# context_items = [pages_and_chunks[i] for i in indices]
context_items = []
for result in search_results:
    context_items.append({"sentence_chunk": result.payload['text']})

# Format prompt with context items
prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)

Query: What is the function of rviz in ROS?
<|im_start|>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

Example 2:
Query: What are the causes of type 2 diabetes?
An

In [58]:
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate an output of tokens
outputs = llm_model.generate(**input_ids,
                             temperature=0.7, # lower temperature = more deterministic outputs, higher temperature = more creative outputs
                             do_sample=True, # whether or not to use sampling, see https://huyenchip.com/2024/01/16/sampling.html for more
                             max_new_tokens=256) # how many new tokens to generate from prompt 

# Turn the output tokens into text
output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}")
print(f"RAG answer:\n{output_text}").replace(prompt, '')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Query: What is the function of rviz in ROS?
RAG answer:
<s><|im_start|> user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

Example 2:
Query: What are the causes of typ

AttributeError: 'NoneType' object has no attribute 'replace'

In [66]:
def ask(query, 
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    
    # Get just the scores and indices of top related results
    # scores, indices = retrieve_relevant_resources(query=query,
    #                                               embeddings=embeddings)
    search_results = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)
    
    # Create a list of context items
    # context_items = [pages_and_chunks[i] for i in indices]
    context_items = []
    for result in search_results:
        context_items.append({"sentence_chunk": result.payload['text']})

    scores = [result.score for result in search_results]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i] # return score back to CPU 
        
    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    
    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    # Only return the answer without the context items
    if return_answer_only:
        return output_text
    
    return output_text, context_items

In [68]:
# Define helper function to print wrapped text 
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
query = random.choice(query_list)
print(f"Query: {query}")

# Answer query with context and return context 
answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Query: What is the purpose of rqt_graph, and how can it help in debugging a ROS system?
Answer:



NameError: name 'print_wrapped' is not defined

In [69]:
print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Answer:

<s><|im_start|> user Based on the following context items, please answer the
query. Give yourself room to think by extracting relevant passages from the
context before answering the query. Don't return the thinking, only return the
answer. Make sure your answers are as explanatory as possible. Use the following
examples as reference for the ideal answer style.  Example 1: Query: What are
the fat-soluble vitamins? Answer: The fat-soluble vitamins include Vitamin A,
Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats
in the diet and can be stored in the body's fatty tissue and liver for later
use. Vitamin A is important for vision, immune function, and skin health.
Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E
acts as an antioxidant, protecting cells from damage. Vitamin K is essential for
blood clotting and bone metabolism.  Example 2: Query: What are the causes of
type 2 diabetes? Answer: Type 2 diabetes is often 

[{'sentence_chunk': "powered by ros. using python and c plus plus. ros is also open source with an. active and growing community so. you can directly see what's going on you. can get help",
  'score': 0.4759140610694885},
 {'sentence_chunk': "and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part",
  'score': 0.40794384479522705},
 {'sentence_chunk': "and a framework built especially for. robotics applications rao's goal. is to provide a standard for robotic. software. that developers can use and reuse in. any robot once you know how to use ros",
  'score': 0.37814757227897644},
 {'sentence_chunk': "so this is the why behind ross. now when to use ros maybe. you've already started to program robots. with. an arduino board with a custom code. on a computer and what you will quickly",
  'score': 0.37660783529281616},
 {'sentence_chunk': "separating your code into re